# Setup

In [ ]:
!pip list

## Installations

In [ ]:
%pip install -r "requirements.txt"

In [ ]:
%pip uninstall -y triton

## Imports

In [35]:
import pandas as pd
import gffpandas.gffpandas as gffpd
from py2neo import Graph, Node, Relationship, cypher
from bisect import bisect_left as bisect

In [36]:
from typing import Sequence, Callable
from random import Random, randrange
import Levenshtein as lev
from multiprocessing.pool import ThreadPool as Pool

In [37]:
import os
import torch
from transformers import AutoTokenizer, AutoModel

/home/daftylooper/Desktop/Capstone/code/finetune-chatgpt-go/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [38]:
import transformers
transformers.__version__

'4.29.2'

In [39]:
def thread_starmap(arg_list, *, threads, progress_step=255):
  def progress(f):
    def inner(i, args):
      if not i&progress_step: print('Invocation', i)
      return f(*args)

  def inner(fn):
    with Pool(threads) as p:
      print('Running map...')
      p.starmap(progress(fn), enumerate(arg_list))
      print('Map complete. Destroying pool')
    print('Pool complete.')
    return fn
  return inner

## Connection

In [197]:
import os
from dotenv import load_dotenv
load_dotenv()

graph = Graph("neo4j+s://"+os.getenv("neo4j_connection_url"), auth=(os.getenv("neo4j_user"), os.getenv("neo4j_password")))

# Check connection
graph.run("RETURN 1")

1
1


# Main Graph

In [ ]:
def add_triple_to_graph(i, triple):

  if not i&255: print('Triple number', i)
      
  h, r, t = triple
  # Create or match the nodes, 'name' is a property
  head_node = Node("Entity", name=h)
  tail_node = Node("Entity", name=t)

  # Merge ensures nodes are only created if they don't already exist
  graph.merge(head_node, "Entity", "name")
  graph.merge(tail_node, "Entity", "name")

  # Create the relationship (head_node)-[relation]->(tail_node)    
  relationship = Relationship(head_node, r, tail_node)

  # Merge the relationship into the graph
  graph.merge(relationship)

## Create Sequence and Identity nodes + relations

In [ ]:
import csv
exon_data = []
path = "./data/exons/exon_sequences.csv"
hgnc = set()
labels = ["gene_id","transcript_id","hgnc_id","gene_type","gene_name","exon_id","protein_id","sequence"]
count = 0
with open(path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        # hgnc.add(tuple(row))
        exon_data.append(dict(zip(labels, row)))

# print(f"number of unique hgnc ids {len(hgnc)}")
exon_data = exon_data[1:]
print(f"pulled {len(exon_data)} sequences!")
exon_data = [kv for kv in exon_data if len(kv)==len(labels)]
print(f"new length {len(exon_data)}")

In [ ]:
# maybe make a decorator for the thread pool thingi?

# TODO: ADD GO ref ids to Identity nodes later
seq_node_id_node_query = '''
MERGE (s:Sequence {sequence: $sequence, exon_id: $exon_id, protein_id: $protein_id})-[:MAPS_TO]->(i:Identity {gene_id: $gene_id, hgnc_id: $hgnc_id, transcript_id: $transcript_id});
'''
def make_seq_node_with_id_node(i, d):
    # arr is of type - [ensembl_id, gene_type, gene_name, hgnc_id, seq]
    if not i&127: print(f"Processed {i} nodes!")
    graph.run(seq_node_id_node_query, **d)
    

In [ ]:
sequence_identity_node_query = '''
MERGE (s:Sequence {sequence: $sequence, exon_id: $exon_id, protein_id: $protein_id})
MERGE (t:Transcript {transcript_id: $transcript_id})
MERGE (g:Gene {gene_id: $gene_id, hgnc_id: $hgnc_id})
MERGE (s)-[:MAPS_TO]->(t)
MERGE (t)-[:PART_OF]->(g);
'''
# identity_node_query = '''
# MERGE (i:Identity {gene_id: $gene_id, hgnc_id: $hgnc_id, transcript_id: $transcript_id})
# MATCH (e:Sequence {exon_id: $exon_id}), (i:Identity {gene_id: $gene_id})
# MERGE (e)-[:MAPS_TO]->(i);
# '''
# sequence_identity_relation_query = '''
# MATCH (e:Sequence {exon_id: $exon_id}), (i:Identity {gene_id: $gene_id})
# MERGE (e)-[:MAPS_TO]->(i);
# '''
count = 0
total = len(exon_data)
def make_seq_node_with_id_node(i, d):
    global count, total
    # arr is of type - [ensembl_id, gene_type, gene_name, hgnc_id, seq]
    if not count&127: print(f"Processed {count}/{total} nodes!")
    graph.run(sequence_identity_node_query, **d)
    count+=1

In [ ]:
print('Creating pool')
with Pool(100) as p:
  print('Running map...')
  p.starmap(make_seq_node_with_id_node, enumerate(exon_data))
  print('Queries complete. Destroying pool')
print('Pool complete.')

## Map Ensembl ID to Go

In [ ]:
import csv

labels = ["gene_id", "go"]
path = "./data/go/ensembl_go_protein_coding.csv"
go_data = []
go = set()
with open(path, "r") as f:
    reader = csv.reader(f)
    for row in reader:
        go_data.append(dict(zip(labels, row)))
        # go.add(row[1])

go_data = go_data[1:]
print(f"pulled {len(go_data)} records!")
# print(f"unique go ids: {len(go)}")

In [ ]:
gene_go_mapper_query = '''
MATCH (s: Gene {gene_id: $gene_id})
MERGE (go: Go {go_id: $go})
MERGE (s)-[:HAS_GO]->(go);
'''
count = 0
total = len(go_data)
def map_gene_to_go(i, d):
    global count, total
    if not count&127: print(f"Processed {count}/{total} nodes!")
    graph.run(gene_go_mapper_query, **d)
    count+=1

In [ ]:
print('Creating pool')
with Pool(100) as p:
  print('Running map...')
  p.starmap(map_gene_to_go, enumerate(go_data))
  print('Queries complete. Destroying pool')
print('Pool complete.')

In [ ]:
import csv

go_triples = []
path = "./data/go/go_triples_protein_coding.csv"
labels = ["go_id", "relation_type", "property"]

with open(path, "r") as f:
    reader = csv.reader(f)
    for row in reader:
        go_triples.append(dict(zip(labels, row)))

go_triples = go_triples[1:]
print(go_triples[0])
print(go_triples[3])
print(f"pulled {len(go_triples)} go triples!")

In [ ]:
count = 0
total = len(go_triples)
add_go_triple_query = '''
MATCH (go:Go {go_id: $go_id})
MERGE (gop:GoProperty {property: $property})
'''
relation = "MERGE (go)-[:{relation_type}]->(gop)"

def add_go_triple_to_graph(i, triple):
    global count, total
    if not count&127 and count!=0: print(f"processed {count}/{total} triples!")
    r = relation.format(relation_type=triple["relation_type"])
    q = add_go_triple_query+r
    graph.run(q, **triple)
    count+=1

In [ ]:
print('Creating pool')
with Pool(100) as p:
  print('Running map...')
  p.starmap(add_go_triple_to_graph, enumerate(go_triples))
  print('Queries complete. Destroying pool')
print('Pool complete.')

## UniprotKB to KG

In [ ]:
# read every line, csv file
# form a dict of every line( key=header, value=whatever we read)
# create a string.format() to add these k-v's into query
# query creates a node from the sequence with relation type being the header
# run query

import csv

path = "./data/uniprotkb/uniprotkb_protein_coding.csv"
t = []
with open(path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        t.append(row)

header = t[0]
t = t[1:]
distinct = set()
uniprot = []

print(f"{len(header)} headers read")
print(f"{len(t)} uniprot data slices")
for row in t:
    distinct.add(tuple(row))
for row in distinct:
    uniprot.append(dict(zip(header, row)))

print(f"found {len(uniprot)} unique sequences!")

print(uniprot[0])

39 headers read
1840 uniprot data slices
found 1840 unique sequences
{'From': 'ENSP00000366513', 'Entry': 'O94985', 'Reviewed': 'reviewed', 'Entry Name': 'CSTN1_HUMAN', 'Protein names': 'Calsyntenin-1 (Alcadein-alpha) (Alc-alpha) (Alzheimer-related cadherin-like protein) (Non-classical cadherin XB31alpha) [Cleaved into: Soluble Alc-alpha (SAlc-alpha); CTF1-alpha (C-terminal fragment 1-alpha)]', 'Gene Names': 'CLSTN1 CS1 KIAA0911', 'Organism': 'Homo sapiens (Human)', 'Length': '981', 'Absorption': '', 'Active site': '', 'Binding site': '', 'Catalytic activity': '', 'Cofactor': '', 'DNA binding': '', 'pH dependence': '', 'Pathway': '', 'Kinetics': '', 'Function [CC]': 'FUNCTION: Postsynaptic adhesion molecule that binds to presynaptic neurexins to mediate both excitatory and inhibitory synapse formation (By similarity). Promotes synapse development by acting as a cell adhesion molecule at the postsynaptic membrane, which associates with neurexin-alpha at the presynaptic membrane (By simi

In [93]:
create_uniprot_index = '''
MATCH (s:Sequence {protein_id: $protein_id})
MERGE (u:Uniprot {uniprot_id: $uniprot_id})
MERGE (s)-[:HAS_UNIPROT]->(u)
'''

create_uniprot_nodes = '''
MATCH (u:Uniprot {uniprot_id: $uniprot_id})
MERGE (up:UniprotProperty {type: $type, property: $property})
MERGE (u)-[:HAS_UNIPROT_PROPERTY]->(up)
'''

# def populate_query(d):
#     kv_str = []
#     for k, v in d.items():
#         if not v or k=="From": continue
#         # kv_str.append(create_uniprot_nodes.replace("KV", f"type: \'{k}\', property: \'{v}\'"))
#         kv_str.append(create_uniprot_nodes.replace("KV", f"type: \'{k}\', property: \'{v}\'"))

#     return kv_str

# res = populate_query(uniprot[0])

count = 0
total = len(uniprot)

def add_uniprot_triple(i, d):
    global count, total
    if not count&127 and count!=0: print(f"processed {count}/{total} data slices")
    graph.run(create_uniprot_index, protein_id=d["From"], uniprot_id=d["Entry"])
    for k,v in d.items():
        if not v or k=="From": continue
        graph.run(create_uniprot_nodes, uniprot_id=d["Entry"], type=k, property=v)
    count+=1

In [94]:
print('Creating pool')
with Pool(100) as p:
  print('Running map...')
  p.starmap(add_uniprot_triple, enumerate(uniprot))
  print('Queries complete. Destroying pool')
print('Pool complete.')

Creating pool
Running map...
processed 128/1840 data slices
processed 256/1840 data slices
processed 384/1840 data slices
processed 512/1840 data slices
processed 640/1840 data slices
processed 768/1840 data slices
processed 896/1840 data slices
processed 1024/1840 data slices
processed 1152/1840 data slices
processed 1280/1840 data slices
processed 1408/1840 data slices
processed 1536/1840 data slices
processed 1664/1840 data slices
processed 1792/1840 data slices
Queries complete. Destroying pool
Pool complete.


# Search Tree

## Retrieving the Genes and their Node ids

In [106]:
id_map = graph.run('MATCH (g:Sequence) RETURN {g: g, id: elementId(g)}')

In [107]:
id_map = id_map.data()

In [108]:
len(id_map)
id_map[:10]

[{'{g: g, id: elementId(g)}': {'id': '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:0',
   'g': Node('Sequence', exon_id='ENSE00001943628', level=12, limit=1.2786107063293457, protein_id='ENSP00000466958', sequence='TACAGGCAGAAAACTACCTGGCACTGTGGGAGACTGAGTATAGGAGAAAGCAGGATGACGATGGGCCTGAATGGTGGTGTGTTTTCCTCCACACAATGTGGCTTTGTCCTCCAAGGTGTCTCACGTGATGGAAGCTGGCTGATCTGACTTGCTTG', thresh=1.2786107063293457)}},
 {'{g: g, id: elementId(g)}': {'id': '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:1',
   'g': Node('Sequence', exon_id='ENSE00003602238', level=11, limit=1.5374518632888794, protein_id='ENSP00000317992', sequence='CTGGTGTCCTGTCTGTCGGAGACGACGGTGTTGGCGGCCGTGCTGCGGCACATCAGCGTGCTGGTGCCCTGCTTCCTGACCTTCCCCAAGCAGTGCCGCATGCTGCTCAAG', thresh=1.465311884880066)}},
 {'{g: g, id: elementId(g)}': {'id': '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:2',
   'g': Node('Sequence', exon_id='ENSE00001688790', level=11, limit=1.5303996801376343, protein_id='ENSP00000368678', sequence='GCCCCACCTTCGCCCCGCTGCCGCCTGTGGCCCCCTTACACT

In [109]:
id_pairs: list[tuple[str, str]] = [(v['id'], v['g']['sequence']) for r in id_map for v in [r.values().__iter__().__next__()]]

print(len(id_pairs), 'pairs')

id_pairs[:5]

19995 pairs


[('4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:0',
  'TACAGGCAGAAAACTACCTGGCACTGTGGGAGACTGAGTATAGGAGAAAGCAGGATGACGATGGGCCTGAATGGTGGTGTGTTTTCCTCCACACAATGTGGCTTTGTCCTCCAAGGTGTCTCACGTGATGGAAGCTGGCTGATCTGACTTGCTTG'),
 ('4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:1',
  'CTGGTGTCCTGTCTGTCGGAGACGACGGTGTTGGCGGCCGTGCTGCGGCACATCAGCGTGCTGGTGCCCTGCTTCCTGACCTTCCCCAAGCAGTGCCGCATGCTGCTCAAG'),
 ('4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:2',
  'GCCCCACCTTCGCCCCGCTGCCGCCTGTGGCCCCCTTACACTGTGCCCAGACGCCCTACGGCTGCTGCCAGGACAATATCACCGCAGCCCGGGGCGTGGGCCTGGCTGGCTGCCCCA'),
 ('4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:3',
  'ATGCTCTACCTGCTGTGCTCCTGGCCGGAGCTGCCCGTCCTGAGCGCCCTGGAGCTGCTAGACTTCAGCTTCCCCGATTGCCACGTAGGCTCCTTCGCCATCAAGTCGCTGCGGAAACTGAC'),
 ('4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:4',
  'ATCGGCTCCGTGGAGCGCTTCATCCACCACGTGAACGCGTGCATGCGGCAGCGGCAGGAGCGGCAGCGGCTGGCGGCCGTGGTGAGCCGCATCGACGCCTACGAGGTGGTGGAAAGCAGCAGCGACGAAGTGGACAAG')]

In [110]:
import os

os.environ["WANDB_DISABLED"] = "true"
model_name = "zhihan1996/DNABERT-2-117M"
path = "./data/dnabert2_model"
tokenizer, model = None, None
tokenizer_path, model_path = path+"/tokenizer/"+model_name, path+"/model/"+model_name

if os.path.isdir(tokenizer_path) and os.path.isdir(model_path):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, trust_remote_code=True)
    model = AutoModel.from_pretrained(model_path, trust_remote_code=True)
else:
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.save_pretrained(tokenizer_path)
    model.save_pretrained(model_path)

# cuda_model = model.to("cuda")

/home/daftylooper/Desktop/Capstone/code/finetune-chatgpt-go/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/daftylooper/Desktop/Capstone/code/finetune-chatgpt-go/.venv/lib/python3.10/site-packages/transformers/modeling_utils.py:446: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longe

## Distance function using DNABERT-2

In [99]:
def dnabert_embedding_mean(dna):
    # inputs = tokenizer(dna, return_tensors = 'pt')["input_ids"].to('cuda')
    inputs = tokenizer(dna, return_tensors = 'pt')["input_ids"]
    hidden_states = model(inputs)[0] # [1, sequence_length, 768]
    # print(inputs.shape)
    # print(hidden_states.shape)

    # embedding with mean pooling
    embedding_mean = torch.mean(hidden_states[0], dim=0)
    # print(embedding_mean.shape) # expect to be 768
    # print(embedding_mean)
    
    # # embedding with max pooling
    # embedding_max = torch.max(hidden_states[0], dim=0)[0]
    # print(embedding_max.shape) # expect to be 768
    
    return embedding_mean.detach()

In [100]:
def dnabert_embedding_distance(a, b):

    # print(f"got -> \n{a}\n{b}")
    try:
        vd1 = memo[a]
    except KeyError:
        print("KeyError: a:", a)
        
    try:
        vd2 = memo[b]
    except KeyError:
        print("KeyError: b:", b)
    
    return float(torch.linalg.vector_norm(vd1-vd2))

## Calculate and memoize embeddings. Store for future sessions.

In [101]:
import numpy as np

def embedding_maker_worker(i, tup):
    id, seq = tup
    with torch.no_grad():
        if not i&127: print(f"{i}: {len(seq)}")
        arr.append(len(seq))
        memo[seq] = dnabert_embedding_mean(seq)

# print(f"first seven - {arr[:8]}")
# print(f"avg seq len {np.mean(arr)}")

In [104]:
import os
import ast
import csv

path = "./data/working/dnabert2_embeddings.csv"
memo = {}
arr = []

if os.path.exists(path):
    print("Found existing dump of embeddings, loading them...")
    with open(path, "r") as f:
        reader = csv.reader(f)
        for row in reader:
            seq, embedding = row
            embedding = torch.tensor(ast.literal_eval(embedding))
            memo[seq] = embedding
else:
    print("Embeddings don't exist, creating...")
    with Pool(200) as p:
      print('Running map...')
      p.starmap(embedding_maker_worker, enumerate(id_pairs[::-1]))
      print('Queries complete. Destroying pool')

    print("Writing Rows...")
    with open(path, "w") as f:
        writer = csv.writer(f)
        for seq, embd in memo.items():
            writer.writerow([seq, embd.tolist()])

print("Done!")

Found existing dump of embeddings, loading them...
Done!


In [ ]:
# !rm "./data/working/dnabert2_embeddings.csv"

In [111]:
memo["CCCAGATCTCTTCAG"]
print(f"size of id_pairs: {len(id_pairs)}")
print(f"size of memo: {len(memo)}")

size of id_pairs: 19995
size of memo: 8472


In [112]:
print(dnabert_embedding_distance(
  "CCCAGATCTCTTCAG",
  "TTTTTATGCCTCATTCTGTGAAAATTGCTGTAGTCTCTTCCAGTTATGAAGAAG",
))

print(len(memo))

6.9764404296875
8472


## Construction

In [ ]:
query = '''

  MATCH (r:Sequence WHERE elementId(r) = $root_id)

  MATCH (n:Sequence WHERE elementId(n) = $n_id)

  SET r.thresh = $thresh

  SET r.limit = $limit

  SET n.level = $level

  CREATE (r)-[:SEARCH_BRANCH {branch_type: $branch_type}]->(n)

'''

In [ ]:
def create_tree_instructions(graph, id_pairs: list[tuple[str, str]], distance: Callable[[Sequence, Sequence], float], r: Random, level = 0) -> tuple[str, str, list[dict]] | None:

  if not id_pairs: return None

  out = []



  l = len(id_pairs)

  root_idx = r.randrange(l)

  root_id, root = id_pairs.pop(root_idx)



  s = sorted(id_pairs, key=lambda g: distance(root, g[1]))

  split_pos = l//2

  if not s: limit = thresh = 0.0

  elif l//2 >= len(s): limit = thresh = distance(root, s[-1][1])

  else:
    print(s)

    distances = [distance(root, g[1]) for g in s]

    thresh = distances[l//2]

    limit = distances[-1]

    # if limit != thresh: print('EXPECTED', thresh, limit)

    split_pos = bisect(distances, thresh)

    # out.append({'id': root_id, 'limit': limit, 'thresh': thresh, 'far_subtree': (distances[split_pos], distances[-1])})



  near_result = create_tree_instructions(graph, s[:split_pos], distance, r, level=level+1)

  if near_result is not None:

    out.extend(near_result[2])

    out.append({

        "branch_type": "NEAR",

        "root_id": root_id,

        "n_id": near_result[0],

        "thresh": thresh,

        "level": level+1,

        "limit": limit,

    })



  far_result = create_tree_instructions(graph, s[split_pos:], distance, r, level=level+1)

  if far_result is not None:

    out.extend(far_result[2])

    out.append({

        "branch_type": "FAR",

        "root_id": root_id,

        "n_id": far_result[0],

        "thresh": thresh,

        "level": level+1,

        "limit": limit,

    })



  print(level, root_id, root[:64])



  return root_id, root, out

In [ ]:
def edit_distance(a: Sequence, b: Sequence) -> float:  # levenshtein distance

  ai = 0

  bi = 0

  d = 0.0

  while ai < len(a) and bi < len(b):

    if a[ai] == b[bi]:  # no edit

      ai += 1

      bi += 1

    elif ai > 0 and a[ai-1] == b[bi]:  # add to a / delete from b

      d += 1

      bi += 1

    elif bi > 0 and a[ai] == b[bi-1]:  # add to b / delete from a

      d += 1

      ai += 1

    else:  # replacement

      d += 1

      ai += 1

      bi += 1



  d += len(a)-ai

  d += len(b)-bi



  return d

In [36]:
# seed = randrange(1<<32)

# seed = 0x509c2232

seed = 0xf67832cb

print(f'seed = 0x{seed:08x}')

r = Random(seed)
     
# root_id, root, kwarg_list = create_tree_instructions(graph, id_pairs.copy(), edit_distance, r)
root_id, root, kwarg_list = create_tree_instructions(graph, id_pairs.copy(), dnabert_embedding_distance, r)

root_id, root

seed = 0xf67832cb


NameError: name 'create_tree_instructions' is not defined

In [ ]:
# Takes 1-2 seconds. *Blazingly Fast*

print('Creating pool')

with Pool(100) as p:

  print('Running map...')

  p.map(

    lambda kwargs: graph.run(query, **kwargs),

    kwarg_list,

  )

  print('Queries complete. Destroying pool')

print('Pool complete.')

## Verification

In [ ]:
def tree_testing(graph, id_pairs: list[tuple[str, str]], distance: Callable[[Sequence, Sequence], float], r: Random, level = 0) -> tuple[str | None, str | None, list[dict]]:

  if not id_pairs: return None, None, []

  out = []



  l = len(id_pairs)

  root_idx = r.randrange(l)

  root_id, root = id_pairs.pop(root_idx)



  s = sorted(id_pairs, key=lambda g: distance(root, g[1]))

  split_pos = l//2

  if not s:

    thresh = 0.0

    limit = 0.0

  elif l//2 >= len(s): limit = thresh = distance(root, s[-1])

  else:

    distances = [distance(root, g[1]) for g in s]

    thresh = distances[l//2]

    limit = distances[-1]

    if limit != thresh: print('EXPECTED', thresh, limit)

    split_pos = bisect(distances, thresh)

    out.append({'id': root_id, 'limit': limit, 'thresh': thresh, 'far_subtree': (distances[split_pos], distances[-1])})



  near_result = tree_testing(graph, s[:split_pos], distance, r, level=level+1)

  out.extend(near_result[2])

  far_result = tree_testing(graph, s[split_pos:], distance, r, level=level+1)

  out.extend(far_result[2])



  if level < 7: print(level, root_id, root[:64])



  return root_id, root, out



seed = 0xf67832cb

print(f'seed = 0x{seed:08x}')

r = Random(seed)

# root_id, root, lev_properties = tree_testing(graph, id_pairs.copy(), lev.distance, r)

root_id, root, lev_properties = tree_testing(graph, id_pairs.copy(), edit_distance, r)

root_id, root

In [ ]:
[prop for prop in lev_properties if len({*prop['far_subtree']}) == 2]

In [ ]:
len(lev_properties)

In [ ]:
root_id

In [ ]:
len(kwarg_list)

In [ ]:
# Check if update was successful

l = [(d['r']['thresh'], d['r']['limit']) for d in graph.run('MATCH  (r:Sequence)-[:SEARCH_BRANCH]->(n) RETURN r, n').data() if d['r']['thresh'] != d['r']['limit']]

print(len(l))

l

In [ ]:
unique_kwargs = {(kwarg['n_id'], kwarg['root_id']) for kwarg in kwarg_list}

len(unique_kwargs)

In [ ]:
for i, kwargs in enumerate(kwarg_list[:15]):

  print('''

    MATCH (r:Gene WHERE elementId(r) = {root_id!r})

    MATCH (n:Gene WHERE elementId(n) = {n_id!r})

    SET r.thresh = {thresh!r}

    CREATE (r)-[:SEARCH_BRANCH {{branch_type: {branch_type!r}}}]->(n)

    RETURN 1

  '''.format(**kwargs))

## Search

In [ ]:
def search(root_id, root_name, tree_thresh, tree_limit, q, qthresh, paths=1, distance=dnabert_embedding_distance) -> tuple[list[tuple[float, str, str]], int]:

  # print("dist->")
  # print(q)
  # print(root_name)
  d = distance(q, root_name)
  # print(f"Distance: {d}, {tree_thresh=}")

  if tree_limit is None:
      if d<=qthresh:
        print(f"logging: limitless! {[(d, root_id, root_name)]}, {paths}")
        return [(d, root_id, root_name)], paths
      else:
        return [], paths
      
  if d > qthresh+tree_limit:
    return [], paths
  
  if tree_limit <= qthresh-d:

    print('SUBTREE BATCH')

    results = graph.run(

      '''

      MATCH (r WHERE elementId(r) = $root_id)-[:SEARCH_BRANCH*1..]->(c)

      RETURN {child_id: elementId(c), child: c.sequence}

      ''',

      root_id=root_id,

    ).data()



    out = []

    for result in results:

      [values] = [*result.values()]

      out.append((distance(q, values['child']), values['child_id'], values['child']))

    if out: print(f"logging: found a substree: {out}")
    return out, paths



  elif d <= qthresh:

    out = [(d, root_id, root_name)]
    print(f"logging, d is less than qthresh: {out}")

  else:

    out = []



  if tree_thresh is None: return out, paths



  result = graph.run(

    '''

    MATCH (r WHERE elementId(r) = $root_id)-[sb:SEARCH_BRANCH]->(c)

    RETURN {child_id: elementId(c), child: c.sequence, thresh: c.thresh, limit: c.limit, branch: sb.branch_type, level: c.level}

    ''',

    root_id=root_id,

  ).data()

  level = None
  children: dict[str, tuple[str, str, str, str]] = {}

  for child in result:
  
    [values] = [*child.values()]
    level = values.get("level", None)
    # print(values)
    children[values['branch']] = values['child_id'], values['child'], values['thresh'], values["limit"]



  if abs(d-tree_thresh) <= qthresh:

    paths += 1

    print(f'Paths: {paths}, Diff: {d-tree_thresh}, Level: {level}')


  # print("near:", children["NEAR"])
  # print("far:", children["FAR"])
    
  if d-qthresh < tree_thresh and 'NEAR' in children:
    # print("NEAR:", children['NEAR'])
    terms, paths = search(*children['NEAR'], q, qthresh, paths)

    out.extend(terms)

  if d+qthresh >= tree_thresh and 'FAR' in children:
    # print("FAR:", children['FAR'])
    terms, paths = search(*children['FAR'], q, qthresh, paths)

    out.extend(terms)



  return (out, paths)

In [38]:
q_result, paths = search(root_id, root, root_thresh, root_limit, root, 30000)

print(len(q_result), 'similar genes')

sorted(q_result)

SUBTREE BATCH
19994 similar genes


[(1.2294673919677734,
  '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:8586',
  'GCCCTGTGCCCTGGCCCTGTTGCGGAGAGACGTGCTGGGGGCCTTCCTGCTGTGGCCTGAGCTGGGTGCTAGCGGCCAGTGGTGTCTGTCCGTGCGCACGCAGTGCGGCGTGGTGCCCCACCAGGTCTTCCGGAACCACCTGGGCCGCTACTGCTTGGAG'),
 (1.2294673919677734,
  '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:9176',
  'GCCCTGTGCCCTGGCCCTGTTGCGGAGAGACGTGCTGGGGGCCTTCCTGCTGTGGCCTGAGCTGGGTGCTAGCGGCCAGTGGTGTCTGTCCGTGCGCACGCAGTGCGGCGTGGTGCCCCACCAGGTCTTCCGGAACCACCTGGGCCGCTACTGCTTGGAG'),
 (1.2393040657043457,
  '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:10485',
  'GAGGTGTCCTGTGGGACCTGGAGAGCCCTCCCGTGTGCCTGACTGTGGGGCCCGGGCCTGTCCGCACCCTGTTGAGCCTGGAGGATGCCGTGTGGGCCAGCTGTGGGCCCCGGGTCACTGTCCTGGAAGCCACCACCCTGCAGCCTCAG'),
 (1.2393040657043457,
  '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:8231',
  'GAGGTGTCCTGTGGGACCTGGAGAGCCCTCCCGTGTGCCTGACTGTGGGGCCCGGGCCTGTCCGCACCCTGTTGAGCCTGGAGGATGCCGTGTGGGCCAGCTGTGGGCCCCGGGTCACTGTCCTGGAAGCCACCACCCTGCAGCCTCAG'),
 (1.2393040657043457,
  '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:8624'

## Searching on exon graph

In [161]:
root_thresh = 1.7634133100509644  # queried from the neo4j console for ({level: 0})
root_limit = 6.264886856079102  # queried from the neo4j console for ({level: 0})
root_id, root = ('4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:1956', 'GCCCCACGTGTGTGCTGAGCAGGAGCTGACCCTGGTGGGCCGCCGCCAGCCGTGCGTGCAGGCCTTAAGCCACACGGTGCCGGTGTGGAAGGCCGGCTGTGGGTGGCAGGCGTGGTGCGTGGGTCATGAGCGGAG')

In [ ]:
q_result, paths = search(root_id, root, root_thresh, root_limit, "CCGATTCCACGACTCTTCTTTGCGGGAGAACATACGATCCGTAACTACCCAGCCACAGTGCATGGTGCTCTGCTGAGTGGGCTGCGAGAAGCGGGAAGAATTGCAGACCAGTTTTTGGGGGCCATGTATACGCTGCCTCGCCAGGCCACACCAGGTGTTCCTGCACAGCAGTCCCCAAGCATGTGAGACAGATGCATTCTAAGGGAAGAGGCCCATGTGCCTGTTTCTGCCATGTAAGGAAGGCTCTTCTAGCAATACTAGATCCCACTGAGAAAATCCACCCTGGCATCTGGGCTCCTGATCAGCTGATGGAGCTCCTGATTTGACAAAGGAGCTTGCCTCCTTTGAATGACCTAGAGCACAGGGAGGAACTTGTCCATTAGTTTGGAATTGTGTTCTTCGTAAAGACTGAGGCAAGCAAGTGCTGTGAAATAACATCATCTTAGTCCCTTGGTGTGTG", 0.1)
print(len(q_result), 'similar genes')
print("results!\n-----------")
print(q_result)
print(paths)

KeyError: a: CCGATTCCACGACTCTTCTTTGCGGGAGAACATACGATCCGTAACTACCCAGCCACAGTGCATGGTGCTCTGCTGAGTGGGCTGCGAGAAGCGGGAAGAATTGCAGACCAGTTTTTGGGGGCCATGTATACGCTGCCTCGCCAGGCCACACCAGGTGTTCCTGCACAGCAGTCCCCAAGCATGTGAGACAGATGCATTCTAAGGGAAGAGGCCCATGTGCCTGTTTCTGCCATGTAAGGAAGGCTCTTCTAGCAATACTAGATCCCACTGAGAAAATCCACCCTGGCATCTGGGCTCCTGATCAGCTGATGGAGCTCCTGATTTGACAAAGGAGCTTGCCTCCTTTGAATGACCTAGAGCACAGGGAGGAACTTGTCCATTAGTTTGGAATTGTGTTCTTCGTAAAGACTGAGGCAAGCAAGTGCTGTGAAATAACATCATCTTAGTCCCTTGG


UnboundLocalError: local variable 'vd1' referenced before assignment

In [ ]:
# change after review
def similarity_search(q):
    global memo
    t_memo = memo.copy()
    memo[q] = dnabert_embedding_mean(q)
    q_result, paths = search(root_id, root, root_thresh, root_limit, q, 0.4)
    memo = t_memo
    return q_result

In [206]:
q = "CCCAGATCTCTTCAG"
# print(memo[q])
similarity_search("CCGATTCCACGACTCTTCTTTGCGGGAGAACATACGATCCGTAACTACCCAGCCACAGTGCATGGTGCTCTGCTGAGTGGGCTGCGAGAAGCGGGAAGAATTGCAGACCAGTTTTTGGGGGCCATGTATACGCTGCCTCGCCAGGCCACACCAGGTGTTCCTGCACAGCAGTCCCCAAGCATGTGAGACAGATGCATTCTAAGGGAAGAGGCCCATGTGCCTGTTTCTGCCATGTAAGGAAGGCTCTTCTAGCAATACTAGATCCCACTGAGAAAATCCACCCTGGCATCTGGGCTCCTGATCAGCTGATGGAGCTCCTGATTTGACAAAGGAGCTTGCCTCCTTTGAATGACCTAGAGCACAGGGAGGAACTTGTCCATTAGTTTGGAATTGTGTTCTTCGTAAAGACTGAGGCAAGCAAGTGCTGTGAAATAACATCATCTTAGTCCCTTG")

Paths: 2, Diff: -0.10643541812896729, Level: 1
Paths: 3, Diff: -0.024843931198120117, Level: 2
Paths: 4, Diff: 0.02799677848815918, Level: 3
Paths: 5, Diff: 0.19676756858825684, Level: 4
Paths: 6, Diff: 0.23884296417236328, Level: 5
Paths: 7, Diff: 0.09969151020050049, Level: 6
Paths: 8, Diff: 0.26479434967041016, Level: 7
Paths: 9, Diff: 0.3211681842803955, Level: 8
Paths: 10, Diff: 0.29506003856658936, Level: 9
Paths: 11, Diff: 0.3091745972633362, Level: 10
Paths: 12, Diff: 0.2909333109855652, Level: 11
Paths: 13, Diff: 0.25397467613220215, Level: 12
Paths: 14, Diff: 0.30879706144332886, Level: 13
Paths: 15, Diff: 0.31386542320251465, Level: 14
Paths: 16, Diff: 0.3209952712059021, Level: 12
Paths: 17, Diff: 0.3395393490791321, Level: 13
Paths: 18, Diff: 0.3275999426841736, Level: 14
Paths: 19, Diff: 0.31045669317245483, Level: 13
Paths: 20, Diff: 0.17925924062728882, Level: 14
Paths: 21, Diff: 0.21452319622039795, Level: 11
Paths: 22, Diff: 0.2835557460784912, Level: 12
Paths: 23, Di

[(0.1695890575647354,
  '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:13995',
  'CCGATTCCACGACTCTTCTTTGCGGGAGAACATACGATCCGTAACTACCCAGCCACAGTGCATGGTGCTCTGCTGAGTGGGCTGCGAGAAGCGGGAAGAATTGCAGACCAGTTTTTGGGGGCCATGTATACGCTGCCTCGCCAGGCCACACCAGGTGTTCCTGCACAGCAGTCCCCAAGCATGTGAGACAGATGCATTCTAAGGGAAGAGGCCCATGTGCCTGTTTCTGCCATGTAAGGAAGGCTCTTCTAGCAATACTAGATCCCACTGAGAAAATCCACCCTGGCATCTGGGCTCCTGATCAGCTGATGGAGCTCCTGATTTGACAAAGGAGCTTGCCTCCTTTGAATGACCTAGAGCACAGGGAGGAACTTGTCCATTAGTTTGGAATTGTGTTCTTCGTAAAGACTGAGGCAAGCAAGTGCTGTGAAATAACATCATCTTAGTCCCTTGGTGTGTG'),
 (0.1695890575647354,
  '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:11952',
  'CCGATTCCACGACTCTTCTTTGCGGGAGAACATACGATCCGTAACTACCCAGCCACAGTGCATGGTGCTCTGCTGAGTGGGCTGCGAGAAGCGGGAAGAATTGCAGACCAGTTTTTGGGGGCCATGTATACGCTGCCTCGCCAGGCCACACCAGGTGTTCCTGCACAGCAGTCCCCAAGCATGTGAGACAGATGCATTCTAAGGGAAGAGGCCCATGTGCCTGTTTCTGCCATGTAAGGAAGGCTCTTCTAGCAATACTAGATCCCACTGAGAAAATCCACCCTGGCATCTGGGCTCCTGATCAGCTGATGGAGCTCCTGATTTGACAAAGGAGCTTGCCTCCTTTGAATGACCTAGAGCACAGGGAGGAACTTGTCCATTAGTTT

In [119]:
similarity_search("GTCTGTGATGAGGAATGGCACCACTACGTCCTCAATGTAGAATTCCCGAGTGTGACTCTCTATGTGGATGGCACGTCCCACGAGCCCTTCTCTGTGACTGAGGATTACCCGCTCCATCCATCCCTCAGCTCGTGGTGGGGGCTTGCTGGCAAG")

Paths: 2, Diff: 0.17660927772521973
Paths: 3, Diff: 0.08273720741271973
Paths: 4, Diff: 0.0156630277633667
Paths: 5, Diff: 0.008596062660217285
Paths: 6, Diff: 0.1760491132736206
Paths: 7, Diff: 0.246209979057312
Paths: 8, Diff: 0.21112418174743652
Paths: 9, Diff: 0.3086996078491211
Paths: 10, Diff: 0.2952260971069336
Paths: 11, Diff: 0.30872398614883423
Paths: 12, Diff: 0.38502436876296997
Paths: 13, Diff: 0.34207940101623535
Paths: 14, Diff: 0.4375401735305786


['4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:8734',
 'ACCCCAACACTCACATCCTCTACGCCTGTGACTCCTGTGGGGACAAGTTTCTGGATGCCAACAGCCTGGCTCAGCATGTGCGAATCCACACAGCCCAGGCACTGGTCATGTTCCAGACAGACGCGGACTTCTATCAGCAGTATGGGCCAGGTGGCACGTGGCCTGCCGGGCAGGTGCTGCAGGCTGGGGAGCTGGTCTTCCGCCCTCGCGACGGGGCTGAGGGCCAGCCCGCACTGGCAGAGACCTCCCCTACAGCTCCTGAATGTCCCCCGCCTGCCGAGTGAGCTGGCGGCCCTTCTGACTGTTTATTTAAGGATGGATGGCACCCTGGAACCGGGAAGGGTGGCCTGTTCCCTAGAGAGAATAAATTGGATTATTTTCT',
 1.1764733791351318]

In [ ]:
print(similarity_search(root))

seqs = ["GGGCCCCCTCATAAATGTGCCTTAATTTTCGCAGATAACAGTGGAATAGACATCATTTTGGGAGTCTTCCCCTTTGTCAGGGAGCTACTCCTTAGAGGGACAGAG",
        "AGCCTGAAGCCGCTGCTGGAGAAGCGCCGGCGCGCGCGCATCAACCAGAGCCTGAGCCAGCTTAAGGGGCTCATCCTGCCGCTGCTGGGCCGGGAG",
        "GTCTGTGATGAGGAATGGCACCACTACGTCCTCAATGTAGAATTCCCGAGTGTGACTCTCTATGTGGATGGCACGTCCCACGAGCCCTTCTCTGTGACTGAGGATTACCCGCTCCATCCATCCAAGATAGAAACTCAGCTCGTGGTGGGGGCTTGCTGGCAAG",
        "GCCCAGTTCAATCTGCTGAGCAGCACCATGGACCAGATGAGCAGCCGCGCGGCCTCGGCCAGCCCCTACACCCCAGAGCACGCCGCCAGCGTGCCCACCCACTCGCCCTACGCACAACCCAGCTCCACCTTCGACACCATGTCGCCGGCGCCTGTCATCCCCTCCAACACCGACTACCCCGGACCCCACCACTTTGAGGTCACTTTCCAGCAGTCCAGCACGGCCAAGTCAGCCACCTGGACG",
        "GCTCGCGGGACCCCTGCTCCAACGTGACCTGCAGCTTCGGCAGCACCTGTGCGCGCTCGGCCGACGGGCTGACGGCCTCGTGCCTGTGCCCCGCGACCTGCCGTGGCGCCCCCGAGGGGACCGTCTGCGGCAGCGACGGCGCCGACTACCCCGGCGAGTGCCAGCTCCTGCGCCGCGCCTGCGCCCGCCAGGAGAATGTCTTCAAGAAGTTCGACGGCCCTTGTG"]

out = []
for x in seqs:
    out.append(similarity_search(x))

for res in out:
    print(res)

Paths: 2, Diff: 0.08228719234466553
Paths: 3, Diff: 0.07570517063140869
Paths: 4, Diff: 0.3529479503631592
Paths: 5, Diff: 0.46616506576538086
Paths: 6, Diff: 0.43332672119140625
Paths: 7, Diff: 0.31277787685394287
Paths: 8, Diff: 0.3483086824417114
Paths: 9, Diff: 0.3337254524230957
Paths: 10, Diff: 0.4161558747291565
Paths: 11, Diff: 0.5772736668586731
Paths: 12, Diff: 0.5513873100280762
Paths: 13, Diff: 0.6479716300964355
[(0.0, '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:1956', 'GCCCCACGTGTGTGCTGAGCAGGAGCTGACCCTGGTGGGCCGCCGCCAGCCGTGCGTGCAGGCCTTAAGCCACACGGTGCCGGTGTGGAAGGCCGGCTGTGGGTGGCAGGCGTGGTGCGTGGGTCATGAGCGGAG'), '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:8734', 'ACCCCAACACTCACATCCTCTACGCCTGTGACTCCTGTGGGGACAAGTTTCTGGATGCCAACAGCCTGGCTCAGCATGTGCGAATCCACACAGCCCAGGCACTGGTCATGTTCCAGACAGACGCGGACTTCTATCAGCAGTATGGGCCAGGTGGCACGTGGCCTGCCGGGCAGGTGCTGCAGGCTGGGGAGCTGGTCTTCCGCCCTCGCGACGGGGCTGAGGGCCAGCCCGCACTGGCAGAGACCTCCCCTACAGCTCCTGAATGTCCCCCGCCTGCCGAGTGAGCTGGCGGCCCTTCTGACTGTTTATTTAAGGATGGATGGCACCCT

In [155]:
traverse_similar_node = '''
MATCH (s:Sequence WHERE elementId(s) = $node_id)
MATCH (s)-[:MAPS_TO]->(t:Transcript)
MATCH (t)-[:PART_OF]->(g:Gene)
MATCH (g)-[:HAS_GO]->(go:Go)
MATCH (go)-[ra]->(gop:GoProperty)
MATCH (s)-[:HAS_UNIPROT]->(u:Uniprot)
MATCH (u)-[rb]->(up:UniprotProperty)
return s, t, g, go, gop, u, up;
'''

def get_prop(node_id):
    res = graph.run(traverse_similar_node, node_id=node_id)
    return res.data()

# remove all redundant data
# note: exons are same??
# note: in go property nodes, include property type. fetching the type from relation is SLOW.
def parse_prop_output(data):
    out = {
    "sequence": set(),
    "exon_id": set(),
    "protein_id": set(),
    "transcript_id": set(),
    "hgnc_id": set(),
    "go_id": set(),
    "gop_property": set(),
    "uniprot_id": set(),
    "up_property": set()
    }

    for slice in data:
        for kv in slice.keys():
            if kv=="up": #  or kv=="gop"
                out[str(kv)+"_property"].add((slice[kv]["property"], slice[kv]["type"])) # applied to uniprot or go properties
            else:
                for k,v in slice[kv].items():
                    if k=="property":
                        out[str(kv)+"_"+str(k)].add(v)
                    if k in out:
                        out[k].add(v)

    for k,v in out.items():
        out[k] = list(out[k])

    return out

In [145]:
import json

json.loads(json.dumps(get_prop(root_id)))

[{'s': {'sequence': 'GCCCCACGTGTGTGCTGAGCAGGAGCTGACCCTGGTGGGCCGCCGCCAGCCGTGCGTGCAGGCCTTAAGCCACACGGTGCCGGTGTGGAAGGCCGGCTGTGGGTGGCAGGCGTGGTGCGTGGGTCATGAGCGGAG',
   'exon_id': 'ENSE00001477190',
   'protein_id': 'ENSP00000348982',
   'limit': 6.264886856079102,
   'thresh': 1.7634133100509644},
  't': {'transcript_id': 'ENST00000356575'},
  'g': {'hgnc_id': 'HGNC:3232', 'gene_id': 'ENSG00000162591'},
  'go': {'go_id': 'GO:0005515'},
  'gop': {'property': 'Binding to a protein.'},
  'u': {'uniprot_id': 'O75095'},
  'up': {'property': 'O75095', 'type': 'Entry'}},
 {'s': {'sequence': 'GCCCCACGTGTGTGCTGAGCAGGAGCTGACCCTGGTGGGCCGCCGCCAGCCGTGCGTGCAGGCCTTAAGCCACACGGTGCCGGTGTGGAAGGCCGGCTGTGGGTGGCAGGCGTGGTGCGTGGGTCATGAGCGGAG',
   'exon_id': 'ENSE00001477190',
   'protein_id': 'ENSP00000348982',
   'limit': 6.264886856079102,
   'thresh': 1.7634133100509644},
  't': {'transcript_id': 'ENST00000356575'},
  'g': {'hgnc_id': 'HGNC:3232', 'gene_id': 'ENSG00000162591'},
  'go': {'go_id': 'GO:0005515'},
 

In [156]:
import json

q = "CCCAGATCTCTTCAG"
def query_sequence(q):
    res_id = similarity_search(q)[0]
    # print(res_id)
    res = get_prop(res_id)
    res = json.loads(json.dumps(res))
    res = parse_prop_output(res)

    return res

In [157]:
res = query_sequence(q)
res

Paths: 2, Diff: 0.5606017112731934
Paths: 3, Diff: 0.6051225662231445
Paths: 4, Diff: 0.033757686614990234


{'sequence': ['GTCGTCTCCCGGCAGTGCAGCTGCCGCTACCGCCGCCCTCTGCCCGCCGGCCCGTCTGTCTACCCCCAGCATGAGCGGCCTGCGCGTCTACAGCACGTCGGTCACCGGCTCCCGCGAA'],
 'exon_id': ['ENSE00001462874'],
 'protein_id': ['ENSP00000270792'],
 'transcript_id': ['ENST00000270792'],
 'hgnc_id': ['HGNC:15568'],
 'go_id': ['GO:0016604',
  'GO:0005737',
  'GO:0070062',
  'GO:0007010',
  'GO:0042995'],
 'gop_property': ['cytoskeleton organization',
  'cytoplasm',
  'GO:0070062',
  'cell projection',
  'A prolongation or process extending from a cell, e.g. a flagellum or axon.',
  'GO:0042995',
  'GO:0005615',
  'The contents of a cell excluding the plasma membrane and nucleus, but including other subcellular structures.',
  'nuclear body',
  'GO:0005737',
  'biological_process',
  'GO:1903561',
  'GO:0006996',
  'GO:0007010',
  'cellular_component',
  'GO:0110165',
  'A vesicle that is released into the extracellular region by fusion of the limiting endosomal membrane of a multivesicular body with the plasma membrane. Extracell

# **Flask Server**

In [159]:
import nest_asyncio
from flask import Flask, request, jsonify
import json

nest_asyncio.apply()

app = Flask(__name__)

@app.route('/')
# maybe list all routes on index, acting as a, well index
def home():
    return "Hello, World!"

@app.route('/sequence', methods=['GET'])
def process_sequence():
    sequence = request.headers.get('sequence')
    if not sequence:
        return jsonify({"error": "Sequence header is missing"}), 400

    result = query_sequence(sequence)
    result_json_str = json.dumps(result)

    return result_json_str, 200, {'Content-Type': 'application/json'}

app.run(port=4000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:4000
Press CTRL+C to quit


Paths: 2, Diff: 0.19522511959075928
Paths: 3, Diff: 0.3726609945297241
Paths: 4, Diff: 0.32413995265960693
Paths: 5, Diff: 0.09543192386627197
Paths: 6, Diff: 0.46836721897125244
Paths: 7, Diff: 0.33478665351867676
Paths: 8, Diff: 0.5755215883255005
Paths: 9, Diff: 0.5681746006011963
Paths: 10, Diff: 0.5839974880218506
Paths: 11, Diff: 0.6109078526496887
Paths: 12, Diff: 0.6747538447380066
Paths: 13, Diff: 0.7430379390716553
Paths: 14, Diff: 0.746055006980896


127.0.0.1 - - [11/Nov/2024 21:47:53] "GET /sequence HTTP/1.1" 200 -


Paths: 2, Diff: 0.3400627374649048
Paths: 3, Diff: 0.5233908891677856
Paths: 4, Diff: 0.4952049255371094
Paths: 5, Diff: 0.21604740619659424
Paths: 6, Diff: 0.5453252792358398
Paths: 7, Diff: 0.4731496572494507
Paths: 8, Diff: 0.7004281282424927
Paths: 9, Diff: 0.6686064004898071
Paths: 10, Diff: 0.7382880449295044
Paths: 11, Diff: 0.7640473246574402
Paths: 12, Diff: 0.8130367398262024
Paths: 13, Diff: 0.8990157842636108
Paths: 14, Diff: 0.862129807472229


...

127.0.0.1 - - [11/Nov/2024 22:28:03] "GET /sequence HTTP/1.1" 200 -


Paths: 2, Diff: 0.37763190269470215
Paths: 3, Diff: 0.27599549293518066
Paths: 4, Diff: 0.16494154930114746
Paths: 5, Diff: 0.12665438652038574
Paths: 6, Diff: 0.14321374893188477
Paths: 7, Diff: 0.08023834228515625
Paths: 8, Diff: 0.1064651608467102
Paths: 9, Diff: 0.042402446269989014
Paths: 10, Diff: 0.12590885162353516
Paths: 11, Diff: 0.07091754674911499
Paths: 12, Diff: 0.09293711185455322
SUBTREE BATCH


127.0.0.1 - - [11/Nov/2024 22:35:31] "GET /sequence HTTP/1.1" 200 -


Paths: 2, Diff: 0.2574063539505005
Paths: 3, Diff: 0.020956754684448242
Paths: 4, Diff: 0.07584071159362793
Paths: 5, Diff: 0.021539926528930664
Paths: 6, Diff: 0.21105146408081055
Paths: 7, Diff: 0.06129443645477295
Paths: 8, Diff: 0.1885286569595337
Paths: 9, Diff: 0.24414277076721191
Paths: 10, Diff: 0.2726384401321411
Paths: 11, Diff: 0.29564136266708374
Paths: 12, Diff: 0.32560843229293823
Paths: 13, Diff: 0.34898555278778076
Paths: 14, Diff: 0.4255368709564209


127.0.0.1 - - [11/Nov/2024 22:35:51] "GET /sequence HTTP/1.1" 200 -
